In [1]:
# check GPU
!nvidia-smi

Sun Jul 11 07:46:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
# import
import numpy
from config import *
from optim import Adam
from models import GruModelReg

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 1.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [4]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = GruModelReg(time_size=24, hidden_size=64, feature_size=5)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [5]:
# dataset
import zipfile, requests, io
import pandas as pd
import numpy
import time

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip'
r = requests.get(data_url)
with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    with z.open('datatraining.txt') as f:
        df = pd.read_csv(f)
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,23.18,27.2720,426.0,721.25,0.004793,1
2,23.15,27.2675,429.5,714.00,0.004783,1
3,23.15,27.2450,426.0,713.50,0.004779,1
4,23.15,27.2000,426.0,708.25,0.004772,1
5,23.10,27.2000,426.0,704.50,0.004757,1


In [6]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 5
del_idx = n_inputs * (n_features + 1)
del_cols = [i for i in range(n_features, del_idx, n_features+1)]
del_cols += [i for i in range(del_idx-1, del_idx+n_features)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var1(t-23),var2(t-23),var3(t-23),var4(t-23),var5(t-23),var1(t-22),var2(t-22),var3(t-22),var4(t-22),var5(t-22),var1(t-21),var2(t-21),var3(t-21),var4(t-21),var5(t-21),var1(t-20),var2(t-20),var3(t-20),var4(t-20),var5(t-20),var1(t-19),var2(t-19),var3(t-19),var4(t-19),var5(t-19),var1(t-18),var2(t-18),var3(t-18),var4(t-18),var5(t-18),var1(t-17),var2(t-17),var3(t-17),var4(t-17),var5(t-17),...,var2(t-8),var3(t-8),var4(t-8),var5(t-8),var1(t-7),var2(t-7),var3(t-7),var4(t-7),var5(t-7),var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t)
24,23.18,27.2720,426.0,721.25,0.004793,23.15,27.2675,429.5,714.00,0.004783,23.15,27.245,426.0,713.500000,0.004779,23.15,27.2,426.0,708.250000,0.004772,23.1,27.2,426.0,704.500000,0.004757,23.100,27.200,419.0,701.000000,0.004757,23.100,27.200,419.0,701.666667,0.004757,23.100,27.200,419.0,699.000000,0.004757,...,27.20,0.0,681.5,0.004728,22.945,27.29,0.0,685.0,0.004728,22.945,27.390,0.0,685.0,0.004745,22.89,27.390,0.0,689.0,0.004730,22.89,27.390,0.0,689.5,0.004730,22.89,27.390,0.0,689.0,0.004730,22.89,27.445,0.0,691.000000,0.004739,22.89,27.500,0.0,688.000000,0.004749,0.0
25,23.15,27.2675,429.5,714.00,0.004783,23.15,27.2450,426.0,713.50,0.004779,23.15,27.200,426.0,708.250000,0.004772,23.10,27.2,426.0,704.500000,0.004757,23.1,27.2,419.0,701.000000,0.004757,23.100,27.200,419.0,701.666667,0.004757,23.100,27.200,419.0,699.000000,0.004757,23.100,27.200,419.0,689.333333,0.004757,...,27.29,0.0,685.0,0.004728,22.945,27.39,0.0,685.0,0.004745,22.890,27.390,0.0,689.0,0.004730,22.89,27.390,0.0,689.5,0.004730,22.89,27.390,0.0,689.0,0.004730,22.89,27.445,0.0,691.0,0.004739,22.89,27.500,0.0,688.000000,0.004749,22.89,27.500,0.0,689.500000,0.004749,0.0
26,23.15,27.2450,426.0,713.50,0.004779,23.15,27.2000,426.0,708.25,0.004772,23.10,27.200,426.0,704.500000,0.004757,23.10,27.2,419.0,701.000000,0.004757,23.1,27.2,419.0,701.666667,0.004757,23.100,27.200,419.0,699.000000,0.004757,23.100,27.200,419.0,689.333333,0.004757,23.075,27.175,419.0,688.000000,0.004745,...,27.39,0.0,685.0,0.004745,22.890,27.39,0.0,689.0,0.004730,22.890,27.390,0.0,689.5,0.004730,22.89,27.390,0.0,689.0,0.004730,22.89,27.445,0.0,691.0,0.004739,22.89,27.500,0.0,688.0,0.004749,22.89,27.500,0.0,689.500000,0.004749,22.79,27.445,0.0,689.000000,0.004710,0.0
27,23.15,27.2000,426.0,708.25,0.004772,23.10,27.2000,426.0,704.50,0.004757,23.10,27.200,419.0,701.000000,0.004757,23.10,27.2,419.0,701.666667,0.004757,23.1,27.2,419.0,699.000000,0.004757,23.100,27.200,419.0,689.333333,0.004757,23.075,27.175,419.0,688.000000,0.004745,23.075,27.150,419.0,690.250000,0.004741,...,27.39,0.0,689.0,0.004730,22.890,27.39,0.0,689.5,0.004730,22.890,27.390,0.0,689.0,0.004730,22.89,27.445,0.0,691.0,0.004739,22.89,27.500,0.0,688.0,0.004749,22.89,27.500,0.0,689.5,0.004749,22.79,27.445,0.0,689.000000,0.004710,22.79,27.500,0.0,685.666667,0.004720,0.0
28,23.10,27.2000,426.0,704.50,0.004757,23.10,27.2000,419.0,701.00,0.004757,23.10,27.200,419.0,701.666667,0.004757,23.10,27.2,419.0,699.000000,0.004757,23.1,27.2,419.0,689.333333,0.004757,23.075,27.175,419.0,688.000000,0.004745,23.075,27.150,419.0,690.250000,0.004741,23.100,27.100,419.0,691.000000,0.004739,...,27.39,0.0,689.5,0.004730,22.890,27.39,0.0,689.0,0.004730,22.890,27.445,0.0,691.0,0.004739,22.89,27.500,0.0,688.0,0.004749,22.89,27.500,0.0,689.5,0.004749,22.79,27.445,0.0,689.0,0.004710,22.79,27.500,0.0,685.666667,0.004720,22.79,27.500,0.0,687.000000,0.004720,0.0


In [7]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)
    
    # check start time
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check duration
    duration = time.time()-start_time
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}, times : {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==   error(RMSE):12.147191423491575
 == train [30, 64, 0.001] model ==   error(RMSE):0.4094040465462296
train-size:1068, val-size:267, test-size:676
best_model => error(rmse) : 0.90, param:[30, 64, 0.001], times : 50.350

fold : 2/10
 == train [30, 64, 0.01] model ==   error(RMSE):2.5762684114584338
 == train [30, 64, 0.001] model ==   error(RMSE):0.120715516494221
train-size:1609, val-size:402, test-size:676
best_model => error(rmse) : 0.32, param:[30, 64, 0.001], times : 77.717

fold : 3/10
 == train [30, 64, 0.01] model ==   error(RMSE):7.277071345090187
 == train [30, 64, 0.001] model ==   error(RMSE):0.4307896997646105
train-size:2150, val-size:537, test-size:676
best_model => error(rmse) : 0.10, param:[30, 64, 0.001], times : 103.412

fold : 4/10
 == train [30, 64, 0.01] model ==   error(RMSE):5.061536302270834
 == train [30, 64, 0.001] model ==   error(RMSE):0.9244774174291379
train-size:2691, 

In [8]:
selected_model.reset_state()
predicted = selected_model.predict(test_cv_X)
if GPU:
    predicted = np.asnumpy(predicted)

def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=1.6309033602168461, std=2.172083666539787
RMSE: mean=0.9564661589126173, std=0.846212648848852
MAE: mean=0.8375555762443858, std=0.85048158808541

[training time]
mean : 170.0607096672058, last:290.0075943470001
